In [1]:
import numpy as np
import pandas as pd
from random import seed
from random import *
import time
from math import *
import math
from itertools import groupby
from collections import defaultdict
import sklearn
from sklearn import preprocessing 

### Data Import

In [2]:
def ConvertToLabel(df,pos):
    label_encoder = preprocessing.LabelEncoder() 
    df[pos]= label_encoder.fit_transform(df[pos]) 
    return df

def chunks(df, n):
    for i in range(0, len(df), n):
        yield df[i:i + n]

### Data Import

In [3]:
X_1 = pd.read_csv("project3_dataset1.txt",header=None, delimiter="\t")
XShape, YShape = X_1.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))
Y_1 = X_1[YShape-1]
X_1 = X_1.iloc[:, :YShape-1]
X_1 = X_1.to_numpy()
Y_1 = Y_1.to_numpy()
X_1 = sklearn.preprocessing.normalize(X_1)

X Shape:569 Y Shape: 31


In [4]:
X_2 = pd.read_csv("project3_dataset2.txt",header=None, delimiter="\t")
XShape, YShape = X_2.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))

Y_2 = X_2[YShape-1]
X_2 = ConvertToLabel(X_2,4)
X_2 = X_2.iloc[:, :YShape-1]
X_2 = X_2.to_numpy()
Y_2 = Y_2.to_numpy()
#X_2 = sklearn.preprocessing.normalize(X_2)

X Shape:462 Y Shape: 10


## Library Implementation

In [5]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics 
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_1, Y_1, test_size=0.2)

In [7]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9473684210526315


#### Data Set 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_2, Y_2, test_size=0.2)

In [9]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5698924731182796


## Scratch Implementation

In [10]:
def dataProcessing(X,Y):
    data = np.column_stack((X, Y))
    np.random.shuffle(data)
    RowCount = math.ceil(len(X)/10)
    output = list(chunks(data,RowCount))
    print('Output Count:'+str(len(output))+' Output X Shape:'+str(len(output[0]))+' Output Y Shape:'+str(len(output[0][0])))
    return output

In [11]:
def findPredictedClass(predicted_labels,test_y):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(predicted_labels)):
        if predicted_labels[i] == 1 and test_y[i] == 1:
            tp += 1    
        if predicted_labels[i] == 0 and test_y[i] == 0:
            tn += 1
        if predicted_labels[i] == 1 and test_y[i] == 0:
            fp += 1
        if predicted_labels[i] == 0 and test_y[i] == 1:
            fn += 1
    try:
        accuracy = (tp + tn)/(tp + tn + fp + fn)
    except:
        accuracy = 0.0
    try:
        precision = tp / (tp + fp)
    except:
        precision = 0.0
    try:
        recall = tp / (tp + fn)
    except:
        recall = 0.0
    try:
        fscore = 2 * (precision * recall)/(precision + recall)
    except:
        fscore = 0.0
    return accuracy,precision,recall,fscore

In [12]:
def calculateGini(left, right, classes):
    n_instances = float(len(left) + len(right))
    gini = 0.0
    score = 0.0
    size = float(len(left))
    if size != 0:
        for class_val in classes:
            tempList = [row[-1] for row in left]
            p = tempList.count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)

    score = 0.0
    size = float(len(right))
    if size != 0:
        for class_val in classes:
            tempList = [row[-1] for row in right] 
            p = tempList.count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)

    return gini

In [21]:
def getSplit(dataset, isRandomForest):
    classes = list(set([row[-1] for row in dataset]))
    indexesList = []

    if isRandomForest:
        indexesList = [x for x in range(len(dataset[0]) - 1)] 
        shuffle(indexesList)
    else:                                               
        indexesList = [x for x in range(len(dataset[0]) - 1)] 
        
    index, value, score, left, right = inf, inf, inf, None, None

    for i in indexesList:
        for row in dataset:
            l, r = [], []
            for d in dataset:
                if d[i] < row[i]: l.append(d)
                else:             r.append(d)
            gini = calculateGini(l, r, classes)
            if gini <= score:
                index, value, score, left, right = i, row[i], gini, l, r
    #print(gini)
    return {'index': index, 'value': value, 'leftHalf': left, 'rightHalf': right}

In [14]:
def terminal(group):
    outcomes = [row[-1] for row in group]
    d = defaultdict(float)
    for i in outcomes:
        d[i] += 1
    result = max(d.items(), key=lambda x: x[1])
    return result[0]

In [15]:
def nodeSplit(node, depth, isRandomForest):
    left, right = node['leftHalf'], node['rightHalf']
    
    del (node['leftHalf'])
    del (node['rightHalf'])

    if not left or not right:
        node['left'] = node['right'] = terminal(left + right)
        return

    if depth >= maxDepth:
        node['left'], node['right'] = terminal(left), terminal(right)
        return

    if len(left) <= minSize:
        node['left'] = terminal(left)
    else:
        node['left'] = getSplit(left, isRandomForest)
        nodeSplit(node['left'], depth + 1, isRandomForest)

    if len(right) <= minSize:
        node['right'] = terminal(right)
    else:
        node['right'] = getSplit(right, isRandomForest)
        nodeSplit(node['right'], depth + 1, isRandomForest)


In [16]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [17]:
def buildTree(train, isRandomForest):
    root = getSplit(train, isRandomForest)
    nodeSplit(root, 1, isRandomForest)
    return root

def buildDecisionTree(train, test):
    tree = buildTree(train, False)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return (predictions), tree

In [18]:
def decisionTrees(output):
    totalAccuracy, totalPrecision, totalRecall, totalFscore = [],[],[],[]
    for i in range(len(output)):
        test_x,test_y,train_x = [], [], []
        train_x = [row for j,lt in enumerate(output) if j!=i for row in lt]
        test_x = output[i]
        for arr in test_x:
            test_y.append(arr[-1])
        predicted_labels, tree = buildDecisionTree(train_x, test_x)
        accuracy, precision, recall, fscore = findPredictedClass(predicted_labels,test_y)
        print('Round '+ str(i) + ' accuracy:' + str(accuracy))
        totalAccuracy.append(accuracy)
        totalPrecision.append(precision)
        totalRecall.append(recall)
        totalFscore.append(fscore)

    avgAccuracy  = np.mean(totalAccuracy)
    avgPrecision = np.mean(totalPrecision)
    avgRecall    = np.mean(totalRecall)
    avgFscore    = np.mean(totalFscore)  
    print('\n')
    print('Avgerage Accuracy :'+ str(avgAccuracy))
    print('Avgerage Precision :'+ str(avgPrecision))
    print('Avgerage Recall :'+ str(avgRecall))
    print('Avgerage Fscore :'+ str(avgFscore))


#### DataSet 1

In [19]:
maxDepth = 3
minSize = 1

In [22]:
output = dataProcessing(X_1, Y_1)
decisionTrees(output)

Output Count:10 Output X Shape:57 Output Y Shape:31
Round 0 accuracy:0.9649122807017544
Round 1 accuracy:0.9824561403508771
Round 2 accuracy:0.9122807017543859
Round 3 accuracy:0.9473684210526315
Round 4 accuracy:0.9122807017543859
Round 5 accuracy:0.8596491228070176
Round 6 accuracy:0.9473684210526315
Round 7 accuracy:0.8947368421052632
Round 8 accuracy:0.9122807017543859
Round 9 accuracy:0.9464285714285714


Avgerage Accuracy :0.9279761904761903
Avgerage Precision :0.9556895059991033
Avgerage Recall :0.8545263476321361
Avgerage Fscore :0.9011597359734858


#### DataSet 2

In [23]:
output = dataProcessing(X_2, Y_2)
decisionTrees(output)

Output Count:10 Output X Shape:47 Output Y Shape:10
Round 0 accuracy:0.8297872340425532
Round 1 accuracy:0.6595744680851063
Round 2 accuracy:0.6808510638297872
Round 3 accuracy:0.8297872340425532
Round 4 accuracy:0.7659574468085106
Round 5 accuracy:0.723404255319149
Round 6 accuracy:0.7021276595744681
Round 7 accuracy:0.7446808510638298
Round 8 accuracy:0.723404255319149
Round 9 accuracy:0.6666666666666666


Avgerage Accuracy :0.7326241134751774
Avgerage Precision :0.6427467140702434
Avgerage Recall :0.5344705075626128
Avgerage Fscore :0.5727717588924486


## Random Forest

In [24]:
def makeSubSample(dataset):
    sample = list()
    size = len(dataset)
    sampleCount = round(size * samplingRatio)
    indexes = [randint(0, size - 1) for x in range(sampleCount)]
    for i in indexes:
        sample.append(dataset[i])
    return sample

In [25]:
def baggingPrediction(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [26]:
def buildRandomForest(train, test, treesCount):
    trees = list()
    for i in range(treesCount):
        sample = makeSubSample(train)
        tree = buildTree(sample, True)
        trees.append(tree)
    predictions = [baggingPrediction(trees, row) for row in test]
    return (predictions)

In [31]:
def randomForest(output):
    for j in [1, 5]:
        totalAccuracy, totalPrecision, totalRecall, totalFscore = [],[],[],[]
        print('Tree Size: ' + str(j))
        for i in range(len(output)):
            test_x,test_y,train_x = [], [], []
            train_x = [row for j,lt in enumerate(output) if j!=i for row in lt]
            test_x = output[i]
            for arr in test_x:
                test_y.append(arr[-1])
            predicted_labels = buildRandomForest(train_x, test_x, j)
            accuracy, precision, recall, fscore = findPredictedClass(predicted_labels,test_y)
            #print('Round '+ str(i) + ' accuracy:' + str(accuracy))
            totalAccuracy.append(accuracy)
            totalPrecision.append(precision)
            totalRecall.append(recall)
            totalFscore.append(fscore)

        avgAccuracy  = np.mean(totalAccuracy)
        avgPrecision = np.mean(totalPrecision)
        avgRecall    = np.mean(totalRecall)
        avgFscore    = np.mean(totalFscore)  
        print('-------------------------------------')
        print('Avgerage Accuracy :'+ str(avgAccuracy))
        print('Avgerage Precision :'+ str(avgPrecision))
        print('Avgerage Recall :'+ str(avgRecall))
        print('Avgerage Fscore :'+ str(avgFscore))
        print('-------------------------------------')

In [32]:
maxDepth = 3
minSize = 1
samplingRatio = 1.0

#### DataSet 1

In [33]:
output = dataProcessing(X_1, Y_1)
randomForest(output)

Output Count:10 Output X Shape:57 Output Y Shape:31
Tree Size: 1
-------------------------------------
Avgerage Accuracy :0.9227443609022554
Avgerage Precision :0.9378499761108458
Avgerage Recall :0.856603809054689
Avgerage Fscore :0.8889866484323148
-------------------------------------
Tree Size: 5
-------------------------------------
Avgerage Accuracy :0.9420112781954886
Avgerage Precision :0.9563317384370016
Avgerage Recall :0.8882599187988598
Avgerage Fscore :0.9181883570715407
-------------------------------------


#### DataSet 2

In [34]:
output = dataProcessing(X_2, Y_2)
randomForest(output)

Output Count:10 Output X Shape:47 Output Y Shape:10
Tree Size: 1
-------------------------------------
Avgerage Accuracy :0.6866884888161484
Avgerage Precision :0.5797137503019856
Avgerage Recall :0.4437049062049062
Avgerage Fscore :0.4661186294729938
-------------------------------------
Tree Size: 5
-------------------------------------
Avgerage Accuracy :0.7152209492635024
Avgerage Precision :0.6233799533799533
Avgerage Recall :0.44980158730158737
Avgerage Fscore :0.5111526375649798
-------------------------------------
